In [10]:
import os
from util import parse_kfold_results
from util import find_optimal_epoch
from util import analyze_convergence
from util import print_recommendations

In [11]:
spe = "yeast1"
models = ["DeepFE-PPI", "DeepTrio", "MyPPI", "PIPR"]
# model2epoch = {"DeepTrio" : 10}
model2epoch = {}
optimal_epochs = {}

for model in models:
    result_file = os.path.join("result", spe, f"{model}.txt")
    if not os.path.exists(result_file):
        continue
    
    with open(result_file, "r", encoding="utf-8") as fin:
        kfold_text = fin.read()

    df = parse_kfold_results(kfold_text)

    epoch_stats, optimal_epoch = find_optimal_epoch(df, model2epoch.get(model, None))
    if optimal_epoch is None:
        continue
    
    optimal_epochs[model] = optimal_epoch

    convergence_df = analyze_convergence(df)
    
    print("=" * 80)
    print(model)
    print("=" * 80)

    print_recommendations(df, epoch_stats, optimal_epoch, convergence_df)

    print("\n")

DeepFE-PPI

最优epoch: 49
--------------------------------------------------
综合评分: 0.9056

平均准确率: 90.31 (±0.28)
平均精确率: 92.39 (±1.30)
平均召回率: 87.90 (±1.35)
平均F1分数: 90.07 (±0.33)
平均ROC AUC: 90.31 (±0.29)
平均PR AUC: 87.24 (±0.70)

平均验证损失: 0.5200

收敛分析:
--------------------------------------------------
 fold  final_f1  stable_epoch  converged_early
    1    0.8973            11             True
    2    0.8930            11             True
    3    0.9041            12             True
    4    0.9056            11             True
    5    0.9034            14             True

前5个最佳epoch:
--------------------------------------------------
 epoch  composite_score  f1_mean  f1_std
    49          0.90560   0.9007  0.0033
    50          0.90522   0.9007  0.0053
    41          0.90434   0.9001  0.0055
    45          0.90372   0.8985  0.0047
    39          0.90354   0.8985  0.0061

建议:
--------------------------------------------------
建议选择epoch 49 - 模型收敛较晚，可能存在过拟合风险
模型表现稳定，各折间方差小


DeepTri

In [12]:
print(f"Model\tAccuracy\tPrecision\tRecall\tF1\tROC AUC\tPR AUC\tComposite Score")
for model, optimal_epoch in optimal_epochs.items():
    accuracy = f"{optimal_epoch['accuracy_mean']*100:.2f}±{optimal_epoch['accuracy_std']*100:.2f}"
    precision = f"{optimal_epoch['precision_mean']*100:.2f}±{optimal_epoch['precision_std']*100:.2f}"
    recall = f"{optimal_epoch['recall_mean']*100:.2f}±{optimal_epoch['recall_std']*100:.2f}"
    f1 = f"{optimal_epoch['f1_mean']*100:.2f}±{optimal_epoch['f1_std']*100:.2f}"
    roc_auc = f"{optimal_epoch['roc_auc_mean']*100:.2f}±{optimal_epoch['roc_auc_std']*100:.2f}"
    pr_auc = f"{optimal_epoch['pr_auc_mean']*100:.2f}±{optimal_epoch['pr_auc_std']*100:.2f}"
    composite_score = f"{optimal_epoch['composite_score']:.4f}"
    

    print(f"{model}\t{accuracy}\t{precision}\t{recall}\t{f1}\t{roc_auc}\t{pr_auc}\t{composite_score}")


Model	Accuracy	Precision	Recall	F1	ROC AUC	PR AUC	Composite Score
DeepFE-PPI	90.31±0.28	92.39±1.30	87.90±1.35	90.07±0.33	90.31±0.29	87.24±0.70	0.9056
DeepTrio	93.81±0.44	96.87±1.66	90.57±0.68	93.60±0.42	93.81±0.46	92.44±1.22	0.9405
MyPPI	95.01±0.90	95.46±1.24	94.51±0.84	94.98±0.91	95.00±0.91	92.96±1.42	0.9500
PIPR	94.90±1.14	97.83±1.75	91.90±3.74	94.71±1.26	94.91±1.14	93.92±0.84	0.9503
